# Demo

In this notebook we show how to use the inference app that is on kubernetes. We will load in random issues from our dataset, and send a post request to the inference endpoint.

In [1]:
import requests
import json
import os
import dotenv
import boto3
import pandas as pd

In [2]:
dotenv.load_dotenv(dotenv.find_dotenv())

True

In [3]:
s3_endpoint_url = os.environ["OBJECT_STORAGE_ENDPOINT_URL"]
s3_access_key = os.environ["AWS_ACCESS_KEY_ID"]
s3_secret_key = os.environ["AWS_SECRET_ACCESS_KEY"]
s3_bucket = os.environ["OBJECT_STORAGE_BUCKET_NAME"]
s3 = boto3.client(
    service_name="s3",
    aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key,
    endpoint_url=s3_endpoint_url,
)

In [4]:
name = os.getenv("REPO_NAME")

if "/" in name:
    REPO = name
    USER = ""
else:
    USER = name
    REPO = ""

In [5]:
savename = USER if USER else REPO.replace("/", "-_-")
path = os.path.join("../data", savename + ".csv")
key = f"github-labeler/data/{savename}.csv"

df = s3.get_object(Bucket=s3_bucket, Key=key)

In [6]:
data = pd.read_csv(df['Body'])

In [7]:
data.sample(5)

,id,repo,title,body,created_at,created_by,num_labels,labels
663,openshift/origin/2443,openshift/origin,Make sure we allow specifying ImageStreamTag a...,"@bparees per our discussion, I'm creating an i...",2015-05-22 13:33:45,soltysh,2,priority/P1\tcomponent/build
6088,openshift/origin/15444,openshift/origin,[ERROR] PID 16377: hack/update-generated-clien...,```\r\nF0724 08:56:58.103443 17314 main.go:2...,2017-07-25 08:27:10,kargakis,2,priority/P3\tkind/test-flake
4820,openshift/origin/12479,openshift/origin,[tests][extended] openshift pipeline build,flake from https://github.com/openshift/origin...,2017-01-13 12:02:41,PI-Victor,3,priority/P2\tcomponent/build\tkind/test-flake
6159,openshift/origin/15632,openshift/origin,Extended.[k8s.io] DNS should provide DNS for t...,The test is failing when running e2e extended ...,2017-08-04 11:02:55,ingvagabund,5,priority/P2\tcomponent/networking\tarea/tests\...
8313,openshift/origin/21667,openshift/origin,Openshift 4 (quick notes on my install failure...,"Sorry if this issue is in the wrong place, I c...",2018-12-15 17:52:55,jayunit100,0,NaN


In [8]:
row = data.sample(1).iloc[0]

In [9]:
print(row['title'])
print('\n - - - - - - \n')
print(row['body'])
print('\n - - - - - - \n')
print(row['labels'])

Router: Provide support for regular-expression route paths

 - - - - - - 

I have [a custom router][origin-router-regexp] based on the default HAProxy-based router that provides regular expression-based path routing. Unfortunately, as described in elyscape/origin-router-regexp#1, #12580 (which was first shipped in v1.5.0-rc3) changed the way that ports are handled. Instead of stripping them entirely, the request's base URI is matched against (in most cases) the output of `generateRouteRegexp $cfg.Host $cfg.Path $cfg.IsWildcard`. [`generateRouteRegexp`][generateRouteRegexp] is a function that builds a regular expression that, in general, follows the following pattern:
```
^${regexp.QuoteMeta($cfg.Host)}(:[0-9]+)?${regexp.QuoteMeta($cfg.Path)}(/.*)?$
```
Translated into words, this matches a string containing precisely the literal hostname, an optional port, the literal path, and optionally any subpaths thereof. This is useful in the general case, but unfortunately breaks my router. It w

In [10]:
post_input = json.dumps(
    {'title': row['title'],
        'body': row['body'],
        'created_by': row['created_by']},
    indent = 0)

In [11]:
r = requests.post(
    'http://github-labeler-ds-github-labeler.apps.smaug.na.operate-first.cloud/predict',
    json = post_input, timeout = 60)

In [12]:
r

<Response [200]>

In [13]:
for x in r.content.decode('utf-8').split('\t'):
    print(x)

component/networking
component/routing
sig/networking
area/documentation
